<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/06_Ajuste_Fino_Clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ajuste Fino (Fine-Tuning) para Clasificación de Texto
Este notebook complementa las diapositivas del curso mostrando un ejemplo práctico de cómo realizar ajuste fino de un modelo preentrenado para una tarea de clasificación de texto.


## 1) Configuración del Entorno

Primero instalamos las bibliotecas necesarias:

In [1]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Importamos las bibliotecas que vamos a utilizar:

In [19]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, concatenate_datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

## 2) Carga de Datos

Para este ejemplo, usaremos el dataset IMDB para análisis de sentimientos, que es público y fácilmente accesible:


In [20]:
# Cargar ejemplos negativos y positivos por separado
negative_examples = load_dataset("imdb", split="train[:500]")  # 500 negativos
positive_examples = load_dataset("imdb", split="train[12500:13000]")  # 500 positivos

# Combinar los datasets
balanced_dataset = concatenate_datasets([negative_examples, positive_examples])

# Verificar la distribución de clases
print("\nDistribución de clases en los datos balanceados:")
labels = balanced_dataset['label']
unique_labels, counts = np.unique(labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"Clase {label} ({'Negativo' if label == 0 else 'Positivo'}): {count} ejemplos ({count/len(labels)*100:.2f}%)")



Distribución de clases en los datos balanceados:
Clase 0 (Negativo): 500 ejemplos (50.00%)
Clase 1 (Positivo): 500 ejemplos (50.00%)


## 3) Preparación del Modelo Base

Utilizaremos un modelo BERT básico como punto de partida:

In [21]:
# Cargamos el modelo base y el tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 4) Preparación de los Datos

Tokenizamos los textos y los preparamos para el entrenamiento:

In [26]:
def tokenize_function(examples):
    return tokenizer(examples["text"],
                    padding="max_length",
                    truncation=True,
                    max_length=512)

print("\nTokenizando datos...")
tokenized_dataset = balanced_dataset.map(tokenize_function, batched=True)
train_test = tokenized_dataset.train_test_split(test_size=0.2, seed=42, shuffle=True)


Tokenizando datos...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Verificar la distribución en los conjuntos de train y test

In [27]:
print("\nDistribución en conjunto de entrenamiento:")
train_labels = train_test["train"]['label']
unique_labels, counts = np.unique(train_labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"Clase {label} ({'Negativo' if label == 0 else 'Positivo'}): {count} ejemplos")

print("\nDistribución en conjunto de test:")
test_labels = train_test["test"]['label']
unique_labels, counts = np.unique(test_labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"Clase {label} ({'Negativo' if label == 0 else 'Positivo'}): {count} ejemplos")



Distribución en conjunto de entrenamiento:
Clase 0 (Negativo): 404 ejemplos
Clase 1 (Positivo): 396 ejemplos

Distribución en conjunto de test:
Clase 0 (Negativo): 96 ejemplos
Clase 1 (Positivo): 104 ejemplos


## 5) Configuración del Entrenamiento

Definimos los parámetros para el ajuste fino:

In [28]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,                  # Aumentado número de épocas
    per_device_train_batch_size=8,       # Reducido para mejor generalización
    per_device_eval_batch_size=8,
    weight_decay=0.005,                  # Reducido weight decay
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",         # Evaluar más frecuentemente
    eval_steps=50,                       # Evaluar cada 50 pasos
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    report_to="none",
    learning_rate=5e-5,                  # Aumentado learning rate
    warmup_ratio=0.1,                    # Añadido warmup
    gradient_accumulation_steps=4,       # Añadido gradient accumulation
    metric_for_best_model="f1",          # Usar F1 para seleccionar mejor modelo
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Definir métricas

In [29]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


## 6) Entrenamiento del Modelo

Realizamos el ajuste fino:

In [30]:
print("\nIniciando entrenamiento...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test["train"],
    eval_dataset=train_test["test"],
    compute_metrics=compute_metrics,
)

print("\nEntrenando el modelo...")
train_result = trainer.train()
print("\nResultados del entrenamiento:")
print(train_result.metrics)


Iniciando entrenamiento...

Entrenando el modelo...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,2.594800,0.545886,0.755000,0.767773,0.757009,0.778846
100,1.341300,0.512044,0.810000,0.802083,0.875000,0.740385



Resultados del entrenamiento:
{'train_runtime': 481.535, 'train_samples_per_second': 8.307, 'train_steps_per_second': 0.26, 'total_flos': 1052444221440000.0, 'train_loss': 2.774471736907959, 'epoch': 5.0}


## 7) Evaluación y Uso del Modelo

Función de predicción

In [31]:
def predict_sentiment(text, threshold=0.6):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    # Preprocesar el texto similar al entrenamiento
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Realizar múltiples forward passes con dropout activo
    model.train()  # Activar dropout
    n_forwards = 5
    all_probs = []

    for _ in range(n_forwards):
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            all_probs.append(probs)

    # Promediar las probabilidades
    avg_probs = torch.mean(torch.stack(all_probs), dim=0)
    neg_prob = avg_probs[0][0].item()
    pos_prob = avg_probs[0][1].item()

    print(f"\nProbabilidades (promediadas sobre {n_forwards} pases):")
    print(f"  Negativo: {neg_prob:.4f}")
    print(f"  Positivo: {pos_prob:.4f}")

    # Usar un umbral más conservador
    if pos_prob > threshold:
        return "Positivo", pos_prob
    elif neg_prob > threshold:
        return "Negativo", neg_prob
    else:
        # Si no estamos seguros, basarnos en la mayor probabilidad
        return "Positivo" if pos_prob > neg_prob else "Negativo", max(neg_prob, pos_prob)

Probamos el modelo con algunos ejemplos:

In [32]:
print("\nProbando el modelo con ejemplos:")
ejemplos = [
    "This movie was absolutely fantastic! Great acting and amazing plot.",
    "Terrible waste of time. The story made no sense and the acting was awful.",
    "It was okay, not great but not terrible either. Somewhat entertaining."
]

print("\nPredicciones:")
print("-" * 60)
for texto in ejemplos:
    print(f"Texto: {texto}")
    sentiment, conf = predict_sentiment(texto)
    print(f"Predicción: {sentiment} (Confianza: {conf:.4f})")
    print("-" * 60)

# Verificar algunos ejemplos del dataset
print("\nEjemplos del dataset:")
for i in range(5):
    print(f"\nTexto {i+1}: {balanced_dataset[i]['text'][:200]}...")
    print(f"Etiqueta: {'Positivo' if balanced_dataset[i]['label'] == 1 else 'Negativo'}")


Probando el modelo con ejemplos:

Predicciones:
------------------------------------------------------------
Texto: This movie was absolutely fantastic! Great acting and amazing plot.

Probabilidades (promediadas sobre 5 pases):
  Negativo: 0.0519
  Positivo: 0.9481
Predicción: Positivo (Confianza: 0.9481)
------------------------------------------------------------
Texto: Terrible waste of time. The story made no sense and the acting was awful.

Probabilidades (promediadas sobre 5 pases):
  Negativo: 0.8970
  Positivo: 0.1030
Predicción: Negativo (Confianza: 0.8970)
------------------------------------------------------------
Texto: It was okay, not great but not terrible either. Somewhat entertaining.

Probabilidades (promediadas sobre 5 pases):
  Negativo: 0.7457
  Positivo: 0.2543
Predicción: Negativo (Confianza: 0.7457)
------------------------------------------------------------

Ejemplos del dataset:

Texto 1: I rented I AM CURIOUS-YELLOW from my video store because of all the 

## 8) Conclusiones y Notas Importantes

En este notebook hemos visto:
1. Cómo cargar un modelo preentrenado
2. Cómo preparar datos para ajuste fino
3. Cómo configurar y realizar el entrenamiento
4. Cómo evaluar y usar el modelo ajustado

Consideraciones importantes:
- Este es un ejemplo simplificado para fines didácticos
- En un caso real, se recomienda:
  - Usar más datos de entrenamiento
  - Realizar una validación cruzada
  - Ajustar hiperparámetros
  - Implementar técnicas de regularización más robustas
  - Considerar el balance de clases

Para adaptar este código a otros problemas de clasificación, puedes modificar:
- El modelo base (según el idioma y dominio)
- Los datos de entrenamiento
- El número de clases (num_labels)
- Los hiperparámetros de entrenamiento